In [1]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -U sentence-transformers

In [3]:
cd '/content/drive/MyDrive/685/catr'

/content/drive/.shortcut-targets-by-id/12c1zkm0_oa8VcOfsUa8Tn_YcYdyPpSlP/685/catr


In [4]:
!pip install -r requirements.txt

In [5]:
from transformers import ViTModel, ViTConfig, ViTFeatureExtractor,BertTokenizer,BertForMaskedLM
#from transformers import DeiTFeatureExtractor, DeiTModel #AutoFeatureExtractor, DeiTForImageClassificationWithTeacher, 
from sentence_transformers import SentenceTransformer
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import auc, precision_score, recall_score,roc_auc_score
import xgboost as xgb
from PIL import Image
import argparse
import glob
import json
import pandas as pd
import numpy as np
from models import caption
from datasets import coco, utils
from tqdm import tqdm
from configuration import Config
from xgboost import XGBClassifier
import os

In [6]:
# data = pd.read_csv('/content/drive/MyDrive/685/emogen/Classifier/train/caption/train_caption.csv')
# from collections import Counter
# Counter(data['personality'].tolist()[:2000])

In [7]:
def extract_vision_transformer_feats(image_path,nsel_st=None,nsel_end=None):
  """
  Function to extract the features from vision transformers
  """
  if not nsel_st:
    img_files_list = glob.glob(image_path+"*")
  else:
    img_files_list = glob.glob(image_path+"*")[nsel_st:nsel_end]
  
  # Create image batch array for Vision Transformer
  img_batch = []
  for file in tqdm(img_files_list):
    img = np.asarray(Image.open(file))
    newsize = (240, 240, 3)
    img = np.resize(img,newsize)
    img_batch.append(img)
  print("Creation of image batches to be used for Vision Transformer complete")

  
  # Extract features from ViTModel
  feature_extractor   = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
  model               = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
  #feature_extractor   =  DeiTFeatureExtractor.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  #model               = DeiTModel.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  inputs              = feature_extractor(images=img_batch, return_tensors="pt")
  outputs             = model(**inputs)

  # Get image representations and their corresponding hashes i.e. get [CLS] token representation for each image
  img_representations = outputs.last_hidden_state[:,0,:]
  img_hash_li         = []
  for file in img_files_list:
    img_hash_li.append(file.split("/")[-1][:-4])
  
  # Create column names for image dimensions 
  col_img  = ["imdim_"+str(i) for i in list(range(768))]

  # Create a dataframe of image features
  img_data = pd.DataFrame(img_representations.detach().numpy(),columns = col_img)
  img_data['img_hash'] = img_hash_li

  return img_data

def extract_sentence_transformer_feats(reference_file_pth = '/content/drive/MyDrive/685/emogen/Classifier/train/caption/',csv_file = 'train_caption.csv',lab_assign=0):
  """
  Extract features for emotion related texts
  """
  # Get reference text data
  combined_path = reference_file_pth + csv_file
  #file = open(combined_path)
  #data_json = json.load(file)
  data = pd.read_csv(combined_path)#pd.DataFrame.from_dict(data_json['annotations']).reset_index(drop=True)
  try:
    del data['Unnamed: 0']
  except:
    pass
  
  # Use Sentence Transformer to extract features
  model = SentenceTransformer('all-mpnet-base-v2')
  sentence_embeddings = model.encode(data['comment'])
  col_text = ["tdim_"+str(i) for i in list(range(768))]

  # Create text feature dataframe
  text_data = pd.DataFrame(sentence_embeddings,columns=col_text)
  text_data['img_hash'] = data['image_hash']
  text_data['label'] = lab_assign

  return text_data

def xgb_train_kfold(X_trn, y_trn,n_splits=5,max_depth=3,n_estimator=150, rand_st=3815):
  """
  Perform training with XGBoost and evaluate in K-Fold cross-validation settings 
  """
  errors    = []
  precision = []
  recall    = []
  auc       = []
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=3815)

  for train_index, test_index in tqdm(kf.split(X_trn)):
     X_train_n, X_test_n = X_trn.values[train_index], X_trn.values[test_index]
     y_train_n, y_test_n = y_trn.values[train_index], y_trn.values[test_index]

     model = XGBClassifier(
         max_depth=max_depth, n_estimators=n_estimator, random_state = rand_st
     )
     model.fit(X_train_n, y_train_n)
     y_pred = model.predict(X_test_n)
     accuracy = (sum(y_pred == y_test_n))/len(y_test_n)
     errors.append(1 - accuracy)
     precision.append(precision_score(y_test_n,y_pred))
     recall.append(recall_score(y_test_n,y_pred))
     auc.append(roc_auc_score(y_test_n,y_pred))

  return errors, precision, recall, auc

In [8]:
img_data = extract_vision_transformer_feats(image_path= '/content/drive/MyDrive/685/emogen/Classifier/train/images/',nsel_st=876,nsel_end=1001)

100%|██████████| 125/125 [00:07<00:00, 16.12it/s]


Creation of image batches to be used for Vision Transformer complete


In [9]:
img_data.to_csv('/content/drive/MyDrive/685/sarcasm/evaluation/Vision_trans/non_sarc_feats/non_sarc_img_feats_1000_batch.csv')

CONCAT NON-SARCASM DATA

In [11]:
%cd '/content/drive/MyDrive/685/sarcasm/evaluation/Vision_trans/non_sarc_feats'
data_200 = pd.read_csv('non_sarc_img_feats_125_batch.csv')
data_400 = pd.read_csv('non_sarc_img_feats_250_batch.csv')
data_600 = pd.read_csv('non_sarc_img_feats_375_batch.csv')
data_800 = pd.read_csv('non_sarc_img_feats_500_batch.csv')
data_1000 = pd.read_csv('non_sarc_img_feats_625_batch.csv')
data_1200 = pd.read_csv('non_sarc_img_feats_750_batch.csv')
data_1400 = pd.read_csv('non_sarc_img_feats_875_batch.csv')
data_1600 = pd.read_csv('non_sarc_img_feats_1000_batch.csv')

/content/drive/.shortcut-targets-by-id/12c1zkm0_oa8VcOfsUa8Tn_YcYdyPpSlP/685/sarcasm/evaluation/Vision_trans/non_sarc_feats


In [12]:
non_sarc_data = pd.concat([data_200,data_400,data_600,data_800,data_1000,data_1200,data_1400,data_1600],axis=0)

In [13]:
non_sarc_data.head()

,Unnamed: 0,imdim_0,imdim_1,imdim_2,imdim_3,imdim_4,imdim_5,imdim_6,imdim_7,imdim_8,imdim_9,imdim_10,imdim_11,imdim_12,imdim_13,imdim_14,imdim_15,imdim_16,imdim_17,imdim_18,imdim_19,imdim_20,imdim_21,imdim_22,imdim_23,imdim_24,imdim_25,imdim_26,imdim_27,imdim_28,imdim_29,imdim_30,imdim_31,imdim_32,imdim_33,imdim_34,imdim_35,imdim_36,imdim_37,imdim_38,...,imdim_729,imdim_730,imdim_731,imdim_732,imdim_733,imdim_734,imdim_735,imdim_736,imdim_737,imdim_738,imdim_739,imdim_740,imdim_741,imdim_742,imdim_743,imdim_744,imdim_745,imdim_746,imdim_747,imdim_748,imdim_749,imdim_750,imdim_751,imdim_752,imdim_753,imdim_754,imdim_755,imdim_756,imdim_757,imdim_758,imdim_759,imdim_760,imdim_761,imdim_762,imdim_763,imdim_764,imdim_765,imdim_766,imdim_767,img_hash
0,0,-0.154419,0.237147,-0.260658,0.079626,0.174629,-0.037283,0.181275,-0.091481,0.075997,0.222657,0.262198,-0.004729,0.085308,0.160205,-0.334296,-0.667960,-0.058677,-0.004470,-0.386030,0.089395,-0.019256,-0.282389,0.417440,-0.034918,0.079087,-0.243365,-0.051581,0.129912,-0.207757,-0.026039,-0.153776,0.213071,-0.000963,-0.242769,-0.015171,-0.316230,0.026008,-0.164314,0.062776,...,-0.326731,-0.385157,0.275468,-0.048805,-0.263003,-0.103545,0.332556,-0.015978,0.230442,-0.355247,-0.195298,0.092949,-0.181343,0.135483,-0.005228,0.659311,0.044282,-0.049064,-0.127675,-0.078598,-0.150954,0.341140,-0.397276,0.167737,-0.202100,0.094768,-0.039785,-0.230607,-0.185947,-0.202292,0.101189,0.060490,0.151284,-0.083219,-0.249288,0.135768,0.111552,0.138786,-0.229597,e17fdeb493691fee90166cd731a1a6
1,1,-0.161675,-0.257757,0.129757,-0.131377,0.103156,0.219500,0.019781,-0.121115,0.002601,0.195443,-0.095846,0.208402,0.151647,0.319363,-0.152885,-0.255986,-0.056179,0.081800,-0.269129,0.215181,-0.110072,-0.174208,0.065126,-0.061155,-0.013843,-0.184014,0.206003,-0.064154,0.366466,-0.198762,-0.439766,-0.055843,-0.113240,-0.324072,0.054468,-0.137400,0.183585,-0.259974,0.030896,...,0.019348,-0.030975,0.266890,0.275151,0.071856,0.150702,0.237042,-0.050178,-0.058340,-0.066797,-0.028146,-0.143062,-0.135885,-0.059849,0.088611,0.093904,0.055797,-0.005112,-0.079915,-0.009697,-0.216545,-0.320368,-0.106363,-0.111194,-0.237161,-0.100384,-0.088738,-0.092409,0.029375,-0.289994,-0.044172,0.216037,-0.033965,-0.155768,-0.063766,0.357158,-0.118469,0.070052,-0.239161,d52c57a634414567e679762f1b3c88e6
2,2,0.131470,-0.335437,0.321686,0.187901,0.015675,-0.128293,0.221141,0.092105,0.246844,0.173362,0.007657,-0.106794,0.466101,-0.154448,0.049409,0.043912,0.018389,-0.153249,-0.140493,-0.135432,0.170132,-0.131044,0.325657,0.023387,-0.173288,0.059496,0.430015,-0.270507,0.180084,0.061132,-0.077753,-0.007201,0.042139,-0.012061,-0.109858,0.359709,-0.403691,0.016272,0.047788,...,-0.105098,-0.076836,0.373948,0.015956,-0.048149,-0.202259,0.194290,0.203501,-0.273842,-0.231286,-0.106424,-0.039620,0.420697,0.117535,-0.049514,0.285343,0.027878,-0.241905,-0.125268,0.131308,-0.164950,0.056566,0.121622,0.070908,0.109604,-0.216308,-0.024494,0.022105,-0.048829,-0.205839,0.006906,-0.036058,-0.149637,-0.104807,0.451444,-0.113600,0.017914,-0.157622,-0.106171,e0ab1f3058ca301884825f31318358fb
3,3,-0.105391,0.213757,-0.336909,-0.043461,-0.173466,0.245793,0.022145,-0.223629,-0.032584,-0.092182,-0.123961,0.006555,0.030328,-0.010635,-0.186472,-0.454358,-0.272072,0.173445,-0.213262,0.174757,-0.017561,0.012228,-0.061416,-0.085261,-0.170512,-0.128842,-0.120785,-0.219215,-0.154824,0.087677,-0.107430,-0.030405,0.194940,-0.141551,-0.277199,0.058098,0.149790,0.136208,0.399637,...,-0.051907,-0.379087,0.156545,0.311087,0.342059,-0.417548,0.138892,-0.130672,-0.256868,-0.180526,-0.363659,0.119540,-0.127883,-0.068358,-0.374897,0.245664,0.154319,-0.272805,-0.224878,0.053213,0.337882,0.003956,-0.240729,-0.137597,-0.006021,0.206694,0.196392,-0.306785,-0.165544,-0.099978,0.051003,0.005749,0.095803,0.045935,-0.348195,-0.129594,-0.199596,0.110170,0.097961,243ebf983476b93dc50b11434f89c1
4,4,-0.031014,0.438352,-0.158522,0.049595,-0.111022,-0.186727,-0.082618,0.039569,0.028709,-0.188519

In [14]:
data = pd.read_csv('/content/drive/MyDrive/685/emogen/Classifier/train/caption/train_caption.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,personality,comment,image_hash,target,label
0,0,0,Envious,living space is heaven,1b6de2b95fae5896b3a4ea3328f72846,not positive,0
1,1,1,Youthful,we should all smile like this girl,b6bdf6d06c411970156241984296825,positive,1
2,2,2,Intelligent,Excellent geologic rock formations,c312ae278f8c7cda60d3fe7d4d861a,not positive,0
3,3,3,Playful,"Oooh, I would love to just cuddle, and chase a...",3ca78427cb573a276685737891a8374,positive,1
4,4,4,Scholarly,This looks like some mid 15th century piece.,2d5d274fdb911ed389e95384e327e0,not positive,0


In [15]:
combined = non_sarc_data.merge(data[['image_hash','personality']],how='left',left_on='img_hash',right_on='image_hash')

In [16]:
combined.columns

Index(['Unnamed: 0', 'imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4',
       'imdim_5', 'imdim_6', 'imdim_7', 'imdim_8',
       ...
       'imdim_761', 'imdim_762', 'imdim_763', 'imdim_764', 'imdim_765',
       'imdim_766', 'imdim_767', 'img_hash', 'image_hash', 'personality'],
      dtype='object', length=772)

In [17]:
del combined['image_hash']

In [18]:
combined2 = combined[combined['personality']!='Witty']

In [19]:
combined2.columns

Index(['Unnamed: 0', 'imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4',
       'imdim_5', 'imdim_6', 'imdim_7', 'imdim_8',
       ...
       'imdim_760', 'imdim_761', 'imdim_762', 'imdim_763', 'imdim_764',
       'imdim_765', 'imdim_766', 'imdim_767', 'img_hash', 'personality'],
      dtype='object', length=771)

In [23]:
del combined2['personality']

In [24]:
combined2.to_csv("/content/drive/MyDrive/685/sarcasm/evaluation/Vision_trans/non_sarc_image_feats.csv",index=False)

In [20]:
comb_text_data = extract_sentence_transformer_feats()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
comb_text_data.to_csv("/content/drive/MyDrive/685/sarcasm/evaluation/Vision_trans/non_sarc_text_feats.csv",index=False)

In [22]:
comb_text_data.columns

Index(['tdim_0', 'tdim_1', 'tdim_2', 'tdim_3', 'tdim_4', 'tdim_5', 'tdim_6',
       'tdim_7', 'tdim_8', 'tdim_9',
       ...
       'tdim_760', 'tdim_761', 'tdim_762', 'tdim_763', 'tdim_764', 'tdim_765',
       'tdim_766', 'tdim_767', 'img_hash', 'label'],
      dtype='object', length=770)

CREATE TEXT FEATS FOR NON-SARCASM DATA

In [ ]:
# data_200 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_200_batch.csv')
# data_400 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_400_batch.csv')
# data_600 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_600_batch.csv')
# data_800 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_800_batch.csv')
# data_1000 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1000_batch.csv')
# data_1200 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1200_batch.csv')
# data_1400 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1400_batch.csv')
# data_1600 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1600_batch.csv')
# data_1740 = pd.read_csv('/content/drive/MyDrive/685/sarcasm/evaluation/img_feats_1740_batch.csv')

In [ ]:
# sarc_data = pd.concat([data_200,data_400,data_600,data_800,data_1000,data_1200,data_1400,data_1600,data_1740],axis=0)

In [ ]:
sarc_data.to_csv('/content/drive/MyDrive/685/sarcasm/evaluation/sarc_feats/only_sarc_image_feats.csv',index=False)

In [ ]:
text_data = extract_sentence_transformer_feats()

In [ ]:
text_data.to_csv('/content/drive/MyDrive/685/sarcasm/evaluation/sarc_feats/sarc_text_feats.csv',index=False)